In [1]:
#Importing the required libraries.
import pandas as pd #for dataframes and reading csv files.
import numpy as np # for math operations(sqrt)
import matplotlib.pyplot as plt #for plotting the graphs
import IPython
from IPython.display import display, HTML
import plotly.express as px
import plotly.graph_objects as go
import csv


%config InlineBackend.figure_format = 'retina'

In [2]:
#displaying dataframes
def display_side(dfs:list, captions:list):
    output = ""
    combined = dict(zip(captions, dfs))
    for caption, df in combined.items():
        output += df.style.set_table_attributes("style='display:inline'").set_caption(caption)._repr_html_()
        output += "\xa0\xa0\xa0"
    display(HTML(output))

In [3]:
#codici_istat_comune
file_name="data/codici_istat_comune.csv"
data_cic=pd.read_csv(file_name,sep=",",delimiter=',',header=0,quoting=3,escapechar="\"",encoding="latin-1")
codist_com=pd.DataFrame(data_cic)
codist_com

#codici_nazioni
file_name="data/codici_nazioni.csv"
data_cin=pd.read_csv(file_name,delimiter=',',header=0,quoting=2, #quoting=2 "Non-Numerical value ex. "Colombia";
                                error_bad_lines=True,encoding="utf-16",quotechar = '"',escapechar='\r')
codist_naz=pd.DataFrame(data_cin)
codist_naz

#disitinct userr day
file_name="data/distinct_users_day.csv"
read_users_day=pd.read_csv(file_name,sep=",",encoding="latin-1")
dist_users_day=pd.DataFrame(read_users_day)
dist_users_day

# Getting DataFrame for codici_istat_provincia.csv
file_name="data/codici_istat_provincia.csv"
data_cip=pd.read_csv(file_name,sep=",",header=0,quoting=3,escapechar="\"",encoding="latin-1")
codist_prov=pd.DataFrame(data_cip)
codist_prov

#dataframe from day_od.csv: "day origin and destination"
file_name="data/day_od.csv"
read_day_od=pd.read_csv(file_name,sep=",",encoding='utf-16', header=0) #comment: explicit header=0 to replace column names;  
columns=["Month","Day","Origin","Destination","Cust_Class","Cod_Country","Cod_Pro","Pro_Com","Flow"]
read_day_od.columns=columns
day_od=pd.DataFrame(read_day_od)

# Punto 2

In [6]:
#Cleaning data and groupbying
visitors = dist_users_day[dist_users_day['CUST_CLASS'] == 'foreigner'
                         ].merge(codist_naz,on='COD_COUNTRY'
                         ).sort_values(by=['VISITORS'],ascending=False,ignore_index=True
                         ).drop(columns=['COD_PRO','PRO_COM','CUST_CLASS'],axis=1
                         ).dropna(axis='rows')
        
visitors1 = visitors.groupby(['COUNTRY_NAME_IT'],as_index=False
                            ).sum(
                            ).sort_values(by=['VISITORS'],ascending=False,ignore_index=True
                            ).drop(columns=['COD_COUNTRY'],axis=1
                            ).copy()
    
display_side([visitors.head(10), visitors1.head(10)], ['Cleaning', 'Groupby'])

,DOW,COD_COUNTRY,VISITORS,COUNTRY_NAME_IT
0,Mercoledì,208.000000,20312,Francia
1,Domenica,228.000000,20264,Svizzera
2,Lunedì,208.000000,19696,Francia
3,Sabato,228.000000,18260,Svizzera
4,Giovedì,208.000000,18164,Francia
5,Domenica,208.000000,18064,Francia
6,Martedì,208.000000,18000,Francia
7,Lunedì,228.000000,17984,Svizzera
8,Sabato,208.000000,17928,Francia
9,Venerdì,208.000000,17508,Francia


In [323]:
fig = go.Figure()

config = {'displayModeBar': True}

df = visitors1[0:20]
fig = px.pie(df, values='VISITORS', names='COUNTRY_NAME_IT', title='Ranking of foreigner visitors weekly'
             ,hole=.25,color_discrete_sequence=px.colors.sequential.RdBu)

fig.update_traces(textposition='inside',hoverinfo='percent+value', textinfo='percent+label+value', textfont_size=12,
                  marker=dict( line=dict(color='#000000', width=1.5)))
#fig.update_layout(template='plotly_dark')
fig.update_layout(margin=dict(t=50, b=0, l=0, r=0))
fig.show(config=config)

# Comments

1) The graph represents the ranking of foreing visitors during a weekly period. The data was treated as a whole and not considering the single days of the week

2) The flux was not scaled by the n° of habitants of the countries in this graph. It was considered only the total number of foreinger people that passed in Padova and the relative flux.


# Punto 3

In [9]:
day_od.columns = map(str.upper, day_od.columns)#preprocessing

day_od1 = day_od[(day_od['COD_COUNTRY'] == 222) & (day_od['CUST_CLASS'] == 'visitor') & (day_od['COD_PRO'] != -999.0)
                ].merge(codist_prov,how='left',on='COD_PRO'
                ).drop(columns=['COD_REG','PROV_SIGLA','COD_COUNTRY','PRO_COM','ORIGIN','DESTINATION']
                ).dropna(axis='rows'       
                ).sort_values(by=['FLOW','PROVINCIA'],ascending=False, na_position='last',ignore_index=True
                ).copy()

day_od2 = day_od1.groupby(['PROVINCIA','MONTH','DAY'],as_index=False
                        ).sum(
                        ).sort_values(by='FLOW',ascending=False,ignore_index=True
                        ).drop(columns=['COD_PRO']
                        )

display_side([day_od1.head(10),day_od2.head(10)],['Cleaning','Provincies'])

,MONTH,DAY,CUST_CLASS,COD_PRO,FLOW,PROVINCIA
0,Aprile,Sabato,visitor,28.000000,1330,Padova
1,Marzo,Venerdì,visitor,28.000000,1282,Padova
2,Maggio,Sabato,visitor,28.000000,1278,Padova
3,Maggio,Lunedì,visitor,28.000000,1273,Padova
4,Marzo,Venerdì,visitor,28.000000,1249,Padova
5,Marzo,Mercoledì,visitor,28.000000,1237,Padova
6,Marzo,Sabato,visitor,28.000000,1229,Padova
7,Maggio,Mercoledì,visitor,28.000000,1226,Padova
8,Aprile,Sabato,visitor,28.000000,1220,Padova
9,Marzo,Giovedì,visitor,28.000000,1216,Padova


In [276]:
filename=r"data/abitanti2.txt"
abitanti = pd.read_csv(filename, header=None,quoting=2,encoding="latin-1")
abitanti = pd.DataFrame(abitanti[0].str.split('\t' ,n=1).tolist())
abitanti.columns = ["PROVINCIA", "VALUE"]
abitanti['VALUE'] = abitanti['VALUE'].apply(pd.to_numeric).sort_values(ascending=False)



day_od3 = day_od2.merge(abitanti,on='PROVINCIA')
day_od3['RATIO'] = 4*day_od3['FLOW']/day_od3['VALUE']
day_od3.sort_values(by=['RATIO','PROVINCIA'],ascending=False)

day_od_total = day_od3.groupby(['PROVINCIA'], as_index=False)['RATIO'].sum(
).sort_values(by='RATIO',ascending=False,ignore_index=True)

display_side([abitanti.head(10),day_od3.head(10),day_od_total.head(10)]
             ,['N° of inhabiants','Flow/Inhabitants','Total Flux'])


,PROVINCIA,VALUE
0,Torino,2269120.000000
1,Vercelli,172307.000000
2,Novara,369595.000000
3,Cuneo,588559.000000
4,Asti,215884.000000
5,Alessandria,424174.000000
6,Biella,177067.000000
7,Verbano-Cusio-Ossola,159159.000000
8,Valle d'Aosta,126202.000000
9,Imperia,214305.000000


In [337]:
fig = go.Figure()

config = {'displayModeBar': True}

df = day_od_total[0:20]
fig = px.pie(df, values='RATIO', names='PROVINCIA', title='Ranking of provinces with most visitors by the total flux'
             ,hole=.25,color_discrete_sequence=px.colors.sequential.RdBu)

fig.update_traces(textposition='inside',hoverinfo='percent+label+value'
                  ,textinfo='percent+label', textfont_size=12
                  ,marker=dict( line=dict(color='#010000', width=1.5))
                  ,pull=[0.2, 0.1, 0, 0]
                  )
#fig.update_layout(template='plotly_dark')
fig.update_traces( selector=dict(type='pie'))
fig.update_layout(margin=dict(t=50, b=0, l=0, r=0))
fig.show(config=config)

In [341]:
fig = go.Figure()
fig = make_subplots(
    rows=2, cols=1, subplot_titles=("Plot 1", "Plot 2")
)
#config = dict({'scrollZoom': True})
config = {'displayModeBar': True,}


data = day_od3

#defining graph
fig = px.bar(data[0:566], x="RATIO", y="PROVINCIA", color='DAY', barmode='stack'
             ,title="Ranking of provinces with most visitors montly"
             ,category_orders={'MONTH':['Febbraio','Marzo','Aprile','Maggio']
             ,'DAY':['Lunedì','Martedì','Mercoledì','Giovedì','Venerdì','Sabato','Domenica']}
             ,orientation='h'
             ,labels={"RATIO": "","PROVINCIA": "",}
             ,hover_name="PROVINCIA"
             ,log_x=True
             #,color_discrete_sequence=['light blue', "blue", "purple", "red"]
             ,animation_frame="MONTH"
             #, animation_group="MONTH"
            )


#defining graph style
fig.update_traces(textposition='outside')
fig.update_layout(uniformtext_minsize=5
                  ,template='ggplot2'
                  ,uniformtext_mode='hide'
                  ,xaxis_range=[-3,1]
                  ,width=900
                  ,height=700
                  ,autosize=False
                 )

fig.update_yaxes(categoryorder='total ascending', tickangle=-25,visible=True)
fig.update_xaxes(fixedrange=True,mirror='all',tickwidth=1,side='top')
fig.show(config=config)

# Commenti

### First Graph
1) The flux is scaled on the n° of habitants per province

2) Since this graph takes into account the total flux ratio and not montly or daily data, so here anomalies like 'Ascoli Piceno' province below aren't relevant

3) About 93% of the total flux is given by visitors living in Veneto


### Second Graph
1) The flux is scaled on the n° of habitants per province

2) The graph has a logarithmic scale due to visualization needs.

3) The first province is indeed Padova, then follow the nearby provincies as expected. Also, there's an important flux of visitors coming from Lombardia region (Milano, Mantova, Varese, Brescia, Bergamo, Monza)

4) It is visible a general progressive increase in the flux from February to April and a decrease in the weekend days as expected.

5) Ascoli Piceno province does not have all data regarding months and days, despite this we see an unusual important flux with a peak on satudays of February. 

6) The ranking of the first 13 provinces remains constant during all the months period, while for the rest there are some slight changes.